# Think Bayes: Chapter 11

This notebook presents code and exercises from Think Bayes, second edition.

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT

In [1]:
from __future__ import print_function, division

%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from itertools import product
from IPython.display import display
from scipy import stats
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
import mythinkbayes as mtb
import numpy as np
import pandas as pd
import pdb

'Done'

'Done'

## Revisiting the Euro Problem

```
A statistical statement appeared in "The Guardian" on Friday January 4. 2002:

When spun on edge 250 times, a Belgian one-euro coin came up heads 140 times and tails 110. "It looks very suspicious to me," said Barry Blight, a statistics lecturer at the London School of Economics. "If the coin were unbiased, the chance of getting a result as extreme as that would be less than 7%."

But do these data give evidence that the coin is biased rather than fair?
```

### What does it mean to have data in favor of a hypothesis?

In chapter 4, Downey proposes that *data are in favor of a hypothesis* if the Bayes factor is greater than 1. 

Reviewing the Bayes factor and its role for determining if data give evidence:

Bayes theorem in probability form:

$P(H | D) = \frac{P(H)P(D|H)}{P(D)}$

For two hypotheses $A$ and $B$, the ratio of posterior probabilities:

$\frac{P(A | D)}{P(B | D)} = \frac{P(A)P(D|A)}{P(B)P(D|B)}$, where the normalizing constants $P(D)$ cancel out.

Assuming $A$ and $B$ are mutually exclusive and collectively exhaustive, then:

$P(B) = 1 - P(A)$, and we can rewrite the ratio of priors and ratio of posteriors as odds:

$O(A|D) = O(A)\frac{P(D|A)}{P(D|B}$, because $O(A) = \frac{P(A)}{P(B)}$

We might say that data favor a hypothesis if the hypothesis is more likely in light of the data than it was without the data. In other words, if the odds of the hypothesis with the data are higher than the odds without, then the data gives evidence for the hypothesis. This is expressed:

$\frac{O(A|D)}{O(A)} = \frac{P(D|A)}{P(D|B)}$, where a result $>1$ indicates that the data $D$ gives evidence for the hypothesis $A$.

The term on the right is the **Bayes Factor**.

To summarize, **Bayes Factor $> 1$ $\rightarrow$ data gives evidence for hypothesis**

In [2]:
class Euro(mtb.PMF):
    
    def likelihood(self, data, hypo):
        p_heads = (hypo / 100)
        p_tails = 1 - p_heads
        n_heads, n_tails = data
        return p_heads**n_heads * p_tails**n_tails

'Done'

'Done'

The likelihood of the 140 heads and 110 tails for an unbiased coin should be ~5.5e-76.

However, this only tells us that the probability of exactly 140 heads and 110 tails is very small.

In [3]:
euro = Euro()
like = euro.likelihood((140, 110), 50)
assert np.isclose(5.5 * 10**-76, like)
print('Likelihood of the data under 50/50 hypothesis =', like)

Likelihood of the data under 50/50 hypothesis = 5.527147875260445e-76


We have the likelihood of an unbiased coin (above), now we need the likelihood of 
a biased coin to compute the Bayes Factor..

$\frac{P(D | B)}{P(D | F)}$, where $B$ and $F$ are the hypotheses that the coin is biased and fair, respectively.

But how do we express $B$?

A naiive option is to compute the likelihood of the exact result (140 heads, 110 tails). 
This yields a high ratio, but it's only useful for this exact result.

In [4]:
euro = Euro()
PDB = euro.likelihood((140, 110), 100 * 140 / 250)
PDF = euro.likelihood((140, 110), 50)
factor = round(PDB / PDF, 2)
assert factor == 6.08
print('Bayes factor, using observed data to define biased hypothesis =', factor)

Bayes factor, using observed data to define biased hypothesis = 6.08


Instead of using the data to define $B$, we need a way to define $B$ objectively.

A better option might be to observe that one side of the coin is more pronounced than the other.
You might suspect that this will cause a 60/40 bias in one direction, but you're not sure which.
Now $B$ consists of two sub-hypotheses, $B_1$ defines a 60/40 bias for heads, $B_2$ defines a 60/40
bias for tails. When computing the likelihood, you take their average to get $B$:

$P(D|B) = 0.5 \times (P(D|B_1) + P(D|B_2))$

In [5]:
euro = Euro()
PDB1 = euro.likelihood((140, 110), 40)
PDB2 = euro.likelihood((140, 110), 60)
PDF = euro.likelihood((140, 110), 50)
factor = round((PDB1 + PDB2) / 2 / PDF, 2)
assert factor == 1.33
print('Bayes factor, using two sub-hypotheses to define biased hypothesis =', factor)

Bayes factor, using two sub-hypotheses to define biased hypothesis = 1.33


To generalize the above approach, we can use a uniform-prior PMF of possible bias values 
and average their likelihoods. 

This yields a small Bayes Factor, about 0.47.

In [6]:
euro = Euro()
biases = mtb.PMF(range(101))
biases[50] = 0
biases.normalize()

PDB = sum([euro.likelihood((140, 110), hypo) * prob for hypo, prob in biases.items()])
PDF = euro.likelihood((140, 110), 50)
factor = round(PDB / PDF, 2)
assert factor == 0.47
print('Bayes factor, where biased hypothesis is an average over biases =', factor)

Bayes factor, where biased hypothesis is an average over biases = 0.47


Now replace the uniform prior for biases with a triangular prior and re-compute the Bayes factor with the same formulation.

In [7]:
euro = Euro()

X = np.arange(0, 101)
m = np.median(X)
P = np.abs(X - m) - m
biases = mtb.PMF(X, P)

PDB = sum([euro.likelihood((140, 110), hypo) * prob for hypo, prob in biases.items()])
PDF = euro.likelihood((140, 110), 50)
factor = round(PDB / PDF, 2)
assert factor == 0.84
print('Bayes factor, where biased hypotheses is an average over biases with a triangular prior =', factor)

Bayes factor, where biased hypotheses is an average over biases with a triangular prior = 0.84


## Moral of the Story: it depends what you mean by *biased*

> In summary, we can use Bayesian hypothesis testing to compare the likelihood of $F$ and $B$,
but we have to do some work to specify precisely what $B$ means. This specification depends on
background information about coins and their behavior when spun, so people could reasonably
disagree about the right definition.

## Exercises

**Exercise 11.1:** Some people believe in the existence of extra-sensory perception (ESP); for example,
the ability of some people to guess the value of an unseen playing card with probability better than
chance.

What is your prior degree of belief in this kind of ESP? Do you think it is as likely to exist as not?
Or are you more skeptical about it? Write down your prior odds.

Now compute the strength of the evidence it would take to convince you that ESP is at least 50% likely
to exist. What Bayes factor would be needed to make you 90% sure that ESP exists?

**Assumptions**

Start with two hypotheses $A$ and $B$, for *ESP doesn't exist* and *ESP does exist*, respectively.

I think ESP is very unlikely to exist. Let's say that my prior belief $P(B)$ is roughly equivalent to the probability of the same American getting struck by lightning once a year for ten consecutive years. [According to the national weather service](http://www.lightningsafety.noaa.gov/odds.shtml), this probability is $(\frac{1}{1,083,000})^{10} \approx 4.5 \times 10^{-61}$. What an unlucky guy!

So $P(B) = 4.5 \times 10^{-61}$, and consequently $P(A) = 1 - 4.5 \times 10^{-61}$

**Strength of evidence to convince me that ESP is at least 50% likely to exist:**

This is equivalent to asking for the strength of evidence such $P(A|D) = P(B|D)$.

Starting with the priors defined above, we should find a way to express the likelihood $P(D|B)$ that represents its "strength"...

$P(B | D) = P(A | D)$

$\rightarrow \frac{P(D|B)P(B)}{P(D)} = \frac{P(D|A)P(A)}{P(D)}$

$\rightarrow P(D|B)P(B) = P(D|A)P(A)$

$\rightarrow P(D|B) = \frac{P(A)}{P(B)} \times P(D|A)$

$\rightarrow P(D|B) = O(A) \times P(D|A)$

So the likelihood $P(D|B$ must be $O(A)$ times greater than the likelihood $P(D|A)$.

$O(A)$ is HUGE: $O(A) = \frac{1 - 4.5\times 10^{-61}}{4.5\times 10^{-61}} \approx 2.2 \times 10^{60}$

**What Bayes factor would be needed to make you 90% sure that ESP exists?**

This is similar to the previous question, except now we need to express the likelihood $P(D|B)$ to satisfy:

$P(B | D) = 9 \times P(A | D)$

$\rightarrow \frac{P(D|B)P(B)}{P(D)} = 9 \times \frac{P(D|A)P(A)}{P(D)}$

$\rightarrow P(D|B)P(B) = 9 \times P(D|A)P(A)$

$\rightarrow P(D|B) = 9 \times \frac{P(A)}{P(B)} \times P(D|A)$

$\rightarrow P(D|B) = 9 \times O(A) \times P(D|A)$

So in this case, $O(A)$ is even HUGER: $O(A) \approx 1.9 \times 10^{61}$


***Note:*** The exercise asked specifically for a Bayes Factor, but I find it more intuitive to think about the evidence as described above.

**Exercise 11.2:** Suppose that your answer to the previous question is 1000; that is, 
evidence with Bayes factor 1000 in favor of ESP would be sufficient to change your mind.

Now suppose that you read a paper in a respectable peer-reviewed scientific journal that
presents evidence with Bayes factor 1000 in favor of ESP. Would that change your mind?

If not, how do you resolve the apparent contradiction?

I would acknowledge that the authors found a particular context in which ESP exists and continue on with my life which consists overwhelmingly of contexts where ESP has not been proven to exist.